In [1]:
import pandas as pd

df = pd.read_csv("googleplaystore.csv")

data = df[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Content Rating']].groupby(['App']).max().dropna().reset_index()
data["Installs"] = pd.to_numeric(data["Installs"].str.replace(",","").str.replace("+",""))
data["Reviews"] = pd.to_numeric(data["Reviews"])
data = data[data["Type"] == "Free"]

q = .8
pop = data[data["Installs"] > data["Installs"].quantile(q)]
pop = pop[pop["Reviews"] > pop["Reviews"].quantile(q)]
pop.sort_values(["Installs","Reviews"], ascending=False)

rev_scores = pop.sort_values(["Installs","Reviews"], ascending=False)
rev_scores = rev_scores.groupby(["Category"]).count().reset_index()

rev_scores["Score"] = round(rev_scores["App"]/min(rev_scores["App"]))

content_scores = pop.sort_values(["Installs","Reviews"], ascending=False)
content_scores = content_scores.groupby(["Content Rating"]).count().reset_index()

C:\Users\AMD\AppData\Local\Temp/ipykernel_15292/2458312762.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Installs"] = pd.to_numeric(data["Installs"].str.replace(",","").str.replace("+",""))


In [2]:
category_scoreboard = rev_scores.iloc[:,[0,1,-1]].sort_values(["App"], ascending=False)
contents = data["Content Rating"].unique()
content_scoreboard = content_scores.iloc[:,:2].sort_values(["App"], ascending=False)
#print( rev_scores.iloc[:,[0,1]])
print(content_scoreboard)

  Content Rating  App
0       Everyone  176
3           Teen   64
1   Everyone 10+   20
2     Mature 17+   14


In [8]:
import ipywidgets as widgets
from IPython.display import display
categories = data["Category"].sort_values().unique()
size = widgets.IntSlider(max = 110)
size_widget = widgets.VBox([widgets.Label("Size [Mb]"), size])
category = widgets.Dropdown(options=categories)
category_widget = widgets.VBox([widgets.Label("Category"), category])
freepaid_button = widgets.ToggleButtons(options=["Free", "Paid"])
content_widget = widgets.RadioButtons(options=["Everyone", "Teen", "Everyone 10", "Mature 17+" ], desciption="Content rating:")

display(size_widget, category_widget, freepaid_button, content_widget)


ToggleButtons(options=('Free', 'Paid'), value='Free')

RadioButtons(options=('Everyone', 'Teen', 'Everyone 10', 'Mature 17+'), value='Everyone')

In [24]:
category_score = 1
category_score = float(category_scoreboard[category_scoreboard["Category"] == category.value]["Score"])
content_score = float ( content_scoreboard[content_scoreboard["Content Rating"] == content_widget.value]["App"] )
size_score = round ( 1/size.value if size.value != 0 else 1, 2)  ## dodać scoreboard

if freepaid_button.value == "Free": 
    subscription_score = 148 # tutaj możnabyłoby przypisać zmienną z analiz
else:
    subscription_score = 1

max_score = category_scoreboard["Score"].max() + content_scoreboard["App"].max() + 1 + 148

score = round (( category_score + size_score + content_score + subscription_score) / max_score , 2) * 100

print("Size score:       ", size_score)
print("Category score:   ", category_score)
print("Free/paid score:  ", subscription_score)
print("Content score:    ", content_score)
print("--------------------------")
print("TOTAL SCORE:      %", score)

Size score:        1
Category score:    93.0
Free/paid score:   148
Content score:     14.0
--------------------------
TOTAL SCORE:      % 61.0
